In [ ]:
# Basic Bitmex Market-Making bot.

import bitmex
import time
import pandas as pd
import datetime as dt

# Enter your own api_key and api_secret. It can be obtained from the bitmex site.
# VERY_IMPORTANT NOTE: keep test = false to trade fakemoney on the bitmex testnet site.
# make test = True to trade using the real money on the bitmex site at your own risk.
# please fully understand the api calls and functions to develop your own algorithm.
# Bitmex api explorer : https://www.bitmex.com/api/explorer/ .

client = bitmex.bitmex(test = False, api_key="XXXXXXXXXXXXX", api_secret="XXXXXXXXXXXXXXXXX")
#client.Order.Order_new(symbol='XBTUSD',orderQty=-1000,price=entry_price-2, ordType='StopLimit',stopPx=entry_price-4).result()
while True:
    if client.Order.Order_getOrders(symbol='XBTUSD',reverse=True, filter = '{"open": "true"}').result()[0] == []:
    orderbook = client.OrderBook.OrderBook_getL2(symbol='XBTUSD', depth = 1).result()
    now = dt.datetime.today()
    start = dt.datetime(now.year,now.month,now.day,now.hour-2,now.minute)
    data = client.Trade.Trade_getBucketed(symbol="XBTUSD", binSize='1m', count=100, startTime = start).result()[0]
    temp_df = pd.DataFrame(data)
    data.head()
    entry_price = orderbook[0][0]['price']
    exit_price = orderbook[0][0]['price'] + 5
    client.Order.Order_new(symbol='XBTUSD',orderQty=1000,price=entry_price).result()
    client.Order.Order_new(symbol='XBTUSD',orderQty=-1000,price=exit_price).result()
    client.Order.Order_new(symbol='XBTUSD',orderQty=2500,stopPx=entry_price+20).result()
    client.Order.Order_new(symbol='XBTUSD',orderQty=-2500,stopPx=entry_price-20).result()
    if client.Order.Order_getOrders(symbol='XBTUSD',reverse=True, filter = '{"open": "true"}').result()[0][0]['ordType'] == 'Stop':
        oid1 = client.Order.Order_getOrders(symbol='XBTUSD',reverse=True, filter = '{"open": "true"}').result()[0][0]['orderID']
        oid2 = client.Order.Order_getOrders(symbol='XBTUSD',reverse=True, filter = '{"open": "true"}').result()[0][1]['orderID']
        client.Order.Order_cancel(orderID=oid1)
        client.Order.Order_cancel(orderID=oid2)
    time.sleep(50)